In [1]:
import xarray as xr
import numpy as np`

In [6]:
input_data = '../era5_700_wind_global_2010_10day.nc'
adjust_coords = True
adjust_time = False
adjust_level = False
adjust_resolution = False
append_additional_data = False
appended_time = None

In [25]:
lon_names = ['longitude', 'lon', 'lons']
lat_names = ['latitude', 'lat', 'lats']
wind_names = ['u', 'v']
level_to_cut = 700

In [30]:
## First, load in the dataset
correct_n_dims = 3
data_xr = xr.open_dataset(input_data)

#### FIRST CHECK THE VARIABLES
### Get a list of the included keys
var_list = list(data_xr.keys())
if (wind_names[0] not in var_list):
    raise Exception("Missing variable "+str(wind_names[0])+" in provided file.")
if (wind_names[1] not in var_list):
    raise Exception("Missing variable "+str(wind_names[1])+" in provided file.")

### CUT DOWN TO JUST THE VARIABLES NEEDED
data_xr = data_xr[wind_names]

### Now, check the dimensions, and adjust if necessary
for var in wind_names:
    n_dims = len(data_xr[var].dims)
    if n_dims < 3:
        raise Exception("Not enough dimensions in dataset. Check that you have at least time, longitude, and latitude included.")
    elif n_dims == 4:
        print('Possibility of uncut level. Check names of dimensions.')

        if 'level' in data_xr[var].dims:
            print('We have a level file, slice!')
            data_xr[var].sel(level = level_to_cut)
        else: 
            raise Exception("More than three dimensions specified, but 'level' not found. Please cut your input data down to just time, latitude, longitude, and level coordinates.")
    elif n_dims > 4:
        raise Exception("Too many coordinates. Please cut your input data down to just time, latitude, longitude, and level (unless already cut to specified level")

    
### Next, check that we have at least a longitude/lon, latitude/lat, and time included. Can do this simultaneously 
lon_key = [i for i in lon_names if lon_names in list(data_xr.dims)]

if adjust_coords == True:
    pass
    
    ## REQUIREMENTS: Latitude goes from positive to negative. Longitude is in -180 to +180 (and not 0 to 360)

In [27]:
data_xr


('time', 'latitude', 'longitude')